네트워크 보안 및 해시레이트 (Network & Mining Data) 활용 전략

비트코인 네트워크의 보안 상태 및 채굴자들의 행동을 분석하여 시장 안정성과 가격 변화를 예측할 수 있습니다.

-> 해시레이트와 난이도 기반 트렌드 분석

지표: hash-rate, difficulty

전략:

해시레이트 상승 → 네트워크 강세 (채굴자들이 장기 보유 예상) → 강세 신호

해시레이트 급락 → 채굴자들이 BTC를 덤핑할 가능성 → 약세 신호


📌 예시: 2021년 중국의 채굴 금지 조치 이후 해시레이트 급락 → BTC 가격 하락.

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_network_data(metric, start_date, end_date):
    """Blockchain Data API에서 네트워크 및 채굴 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 2년 데이터 요청
start_date = datetime.now() - timedelta(days=730)
end_date = datetime.now()

# 가져올 네트워크 관련 데이터
network_metrics = ["hash-rate", "difficulty"]
df_network = fetch_network_data(network_metrics[0], start_date, end_date)

for metric in network_metrics[1:]:
    temp_df = fetch_network_data(metric, start_date, end_date)
    if not temp_df.empty:
        df_network = df_network.merge(temp_df, on="timestamp", how="left")

# 데이터 저장
df_network.to_csv("bitcoin_network_data.csv", index=False)
print("✅ 네트워크 관련 데이터 저장 완료: bitcoin_network_data.csv")

✅ 네트워크 관련 데이터 저장 완료: bitcoin_network_data.csv


3. UTXO 활용 전략

UTXO는 비트코인 보유자들의 행동 패턴을 분석하는 데 중요한 역할을 합니다.

*아래 코드 utxo 총 개수만 읽음*

In [4]:
def fetch_utxo_data(metric, start_date, end_date):
    """Blockchain Data API에서 UTXO 관련 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 가져올 UTXO 관련 데이터
utxo_metrics = ["utxo-count", "total-bitcoins"]
df_utxo = fetch_utxo_data(utxo_metrics[0], start_date, end_date)

# 데이터 저장
df_utxo.to_csv("bitcoin_utxo_data.csv", index=False)
print("✅ UTXO 관련 데이터 저장 완료: bitcoin_utxo_data.csv")

✅ UTXO 관련 데이터 저장 완료: bitcoin_utxo_data.csv
